In [1]:
from __future__ import division, print_function, absolute_import

from tmm.tmm_core import (inc_tmm, unpolarized_RT, ellips,
                       position_resolved, find_in_structure_with_inf)
import numpy as np
from numpy import pi, linspace, inf, array, loadtxt
import math
import cmath
import matplotlib.pyplot as plt

import pandas as pd

from os.path import join

# "5 * degree" is 5 degrees expressed in radians
# "1.2 / degree" is 1.2 radians expressed in degrees
degree = pi/180
DEBUG = False
# Import the function 'join' from the module 'os.path'
# This function automatically takes care of the slashes that need to be added when combining two paths.
dirpath = r'/Users/twenzel/ownCloudP/DreamHost/#Irridescent mineral project/Python simulation TTM/results/'

In [2]:
def profile(n_list, N): #at 600nm
    if not DEBUG:
        return
    position = np.genfromtxt("ProfileTotal.xls",delimiter="\t")
    position = position[:,0]
    pos = np.delete(position, 0)
    poss = np.delete(pos,1)
    #n_list_imaginary = n_list[N/2, 1:-1].imag

    k = 0.002 #500nm
    plt.figure(1)
    #plt.plot(poss, n_list[1:-1].imag, 'red')
    plt.plot(poss, n_list[N/2, 1:-1], 'red')
    
    plt.xlabel('Distance (/nm)')
    plt.ylabel('Im{refractive index}')
    #plt.title('Simulated reflection of unpolarized light at 0$^\circ$ incidence (blue), '
                #'Experimental data (purple)')
    #plt.axis([0, 2000, -0.10, 0.10])
    plt.axis([0, 2000, 1.0, 2.0])
    

In [3]:
def generateNlist(d_list, gray_list,lamb):
    #Normalise gray values to go from 0 (all air) to 1 (solid material)
    minimum = np.amin(gray_list)
    gray_list= gray_list -minimum
    maximum = np.amax(gray_list)
    gray_list = gray_list / maximum
    
    n_list = np.zeros((lamb.size, gray_list.size), dtype=np.complex_)
    n1 = np.zeros((lamb.size, gray_list.size), dtype=np.complex_)
    
    #for k in 1/lamb:
    air = 1.00029
    AS = 0.0162 #Sellmeier parameter for dispersion equation
    BS = 0.2658 #Sellmeier parameter for dispersion equation

    for i in range(len(lamb)):
        k = 1/lamb[i]
        #n=sqrt( (1/( (-AS*(kˆ2))+BS ) ) +1)
        n1[i,:] = math.sqrt( (1/( (-AS*(k**2))+BS ) ) +1)
        
        gray_list_trans=np.zeros((1, gray_list.size))
        gray_list_trans[:, 0:] = gray_list[0:]
        
        for l in range(len(gray_list)):
            # real an imaginary part
            #n_list[i,l].real= air +((n1[i,l].real - air) * (gray_list_trans[0,l]))
            #n_list[i,l].imag= n1[i,l].imag * gray_list_trans[0,l]
            n_list[i,l] = complex(air +((n1[i,l].real - air) * (gray_list_trans[0,l])), n1[i,l].imag * gray_list_trans[0,l])
 
        wavel=1/k
        n1i = n1.imag
        if DEBUG:
            plt.figure(3)
            plt.plot(wavel, ni, marker='o', ms = 10, alpha=1, color='b', label='Rock layers')
            plt.xlabel('Wavelength (/nm)')
            plt.ylabel('Refractive index')
            plt.title('Dispersion relations, imaginary part. Blue = Rock layers')

        
#Adds bottom infinite layer at the end of the n list (same as last n calculated)
    n_last = []
    n_lists = np.column_stack((n_list, n_list[:, -1]))
#Adds air layer at the top of the n list
    n_list = np.insert(n_lists, 0, air, axis=1)

    Filename2 = "n_listreal.txt"
    Filename3 = "n_listimaginary.txt"
    filepathRNA = join(dirpath, Filename2)
    filepathRNA = join(dirpath, Filename3)
    np.savetxt(filepathRNA,  n_list.real, delimiter=" ")
    np.savetxt(filepathRNA, n_list.imag, delimiter=" ")  
    
    return n_list

In [4]:
def reflectionsim(lamb, angles, n_list, d_list, c_list, N):

    RNA = np.zeros((lamb.size,angles.size))
    
    for i in range(lamb.size):
        for j in range(angles.size):
            RNA[i,j] = inc_tmm('s',n_list[i,:], d_list, c_list, angles[j]*degree, lamb[i])['R']
            RNA[i,j] += inc_tmm('p',n_list[i,:], d_list, c_list, angles[j]*degree, lamb[i])['R']
	    #both polarisations, averaged 
    RNA = RNA/2
    RNAmean = np.mean(RNA, axis=1)
    return RNAmean

In [5]:
if __name__ == "__main__":
    main()

NameError: name 'main' is not defined

In [6]:
def main():
    lamb=linspace(400,800,num=100) #40
    angles=linspace(-17,17,35) #19
    N = 1
    
    #Load experimental spectra so they can be plotted together with simulation
    data = np.loadtxt('Spec5.csv',delimiter=';',skiprows=1)
    #data.head() #display first five lines
    expx = np.array(data[:,0])
    expy = np.array(data[:,1])


    RNAmeans = np.zeros((N, lamb.size))

    for n in range(N):
        print(n)
        #Read the layers' positions and gray values from structure profile.
        #ftemp = np.loadtxt("ProfileTotal.csv",delimiter=';',skiprows=1)
        t_list = np.array(data[:,0]) #thickness of each layer
        gray_list = np.array(data[:,1])
        
        #t_list,gray_list = loadGrayvalues("Plot Values "+str(n+20-N)+".xls")
        gray_list = gray_list *2.1823 #refractive index of geothide
        
        #Adds top and bottom inf layers to d list
        t_lists = np.append(t_list, inf)
        d_list = np.insert(t_lists, 0, inf)
        #correction = 0.05 #0.055
        #d_list = d_lists*(1-correction)

        n_list = generateNlist(d_list,gray_list,lamb)
        # C_list: same lenght as t_list. First and last value must be 'i'.
        a = ['c']*len(t_list)
        b = np.append(a, 'i')
        c_list = np.insert(b, 0, 'i')

        RNAmeans[n,:] = reflectionsim(lamb, angles, n_list, d_list, c_list, N)

        plt.figure(0)
        plt.plot(lamb,RNAmeans[n,:],'--')

    plt.plot(lamb,RNAmeans.mean(0),'blue',lw=2,label="Average")
    plt.plot(expx,expy,'purple',lw=2,label="Experimental")
    plt.legend()
    plt.xlabel('Wavelength (/nm)')
    plt.ylabel('Fraction reflected')
    plt.title('Simulated reflection of unpolarized light at 0$^\circ$ incidence (purple), '
                'Experimental data of different sample (purple)')
    plt.axis([400, 800, 0, 0.35])

# Save simulation data in a text file made out of columns delimited by a space
    
    Filename1 = "RNAmeans.txt"
    filepathRNA = join(dirpath, Filename1)
    np.savetxt(filepathRNA, RNAmeans, delimiter=" ")
    
    profile(n_list, N)
    plt.show()